<a href="https://colab.research.google.com/github/cathyxinchangli/ATMS-597-SP-2020/blob/sarah/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Use NCEI API

In [0]:
import requests
import pandas as pd

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    ---
    S. Moline, 'Hands on Data Analysis with Pandas'
    https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/blob/master/ch_04/0-weather_data_collection.ipynb
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'CcXuksWVEIXhFFygXrLJecFbxNfABvYv' # obtained by X. Li, 2/4/2020
        },
        params=payload
    )
def fetch_data(datasetid, datatypeid, startdate, enddate, endpoint="data", datacategoryid="TEMP",
               locationid="FIPS:UK", stationid="GHCND:UK000003026", units="metric", limit=1000):
    response = make_request(endpoint, 
                        {"datasetid": datasetid,
                         "datatypeid": "TMIN",
                         "datacategoryid": datacategoryid,
                         "locationid": locationid,
                         "stationid": stationid,
                         "startdate": datetime.date(1990,6,1),
                         "enddate": datetime.date(1991,6,1),
                         "units": "metric",
                         })
    response = pd.DataFrame(response.json()["results"])
    return response

# This dataset goes between these two dates supposedly
# start = datetime.date(1874, 10, 13)
# end = datetime.date(2020, 2, 4)

In [62]:
# Trying to average max and min values over 1 year
start = datetime.date(2019, 1, 1)
end = datetime.date(2020, 1, 1)
df_max = fetch_data("GHCND", "TMAX", start, end) 
df_min = fetch_data("GHCND", "TMIN", start, end)

df_new = (df_max.add(df_min, axis='value'))

## HAVING TROUBLE DIVIDING "VALUE" COLUMN BY TWO
print(df_new)

                                      date  datatype  ... attributes value
0   1990-06-01T00:00:001990-06-01T00:00:00  TMINTMIN  ...   ,,G,,,G,  21.0
1   1990-06-02T00:00:001990-06-02T00:00:00  TMINTMIN  ...   ,,G,,,G,   9.4
2   1990-06-03T00:00:001990-06-03T00:00:00  TMINTMIN  ...   ,,G,,,G,  19.6
3   1990-06-04T00:00:001990-06-04T00:00:00  TMINTMIN  ...   ,,G,,,G,  18.8
4   1990-06-05T00:00:001990-06-05T00:00:00  TMINTMIN  ...   ,,G,,,G,  18.4
5   1990-06-06T00:00:001990-06-06T00:00:00  TMINTMIN  ...   ,,G,,,G,  12.6
6   1990-06-07T00:00:001990-06-07T00:00:00  TMINTMIN  ...   ,,G,,,G,  15.8
7   1990-06-08T00:00:001990-06-08T00:00:00  TMINTMIN  ...   ,,G,,,G,  14.0
8   1990-06-09T00:00:001990-06-09T00:00:00  TMINTMIN  ...   ,,G,,,G,  18.4
9   1990-06-10T00:00:001990-06-10T00:00:00  TMINTMIN  ...   ,,G,,,G,  18.2
10  1990-06-11T00:00:001990-06-11T00:00:00  TMINTMIN  ...   ,,G,,,G,  18.0
11  1990-06-12T00:00:001990-06-12T00:00:00  TMINTMIN  ...   ,,G,,,G,  18.2
12  1990-06-13T00:00:0019

## Collect All Datapoints
I believe we have to go through a ton of years correct?


In [0]:
import datetime

from IPython import display # for updating the cell dynamically

current = datetime.date(1998, 1, 1)
end = datetime.date(2019, 1, 1)

results = []

while current < end:
    # update the cell with status information
    display.clear_output(wait=True)
    display.display(f'Gathering data for {str(current)}')
    
    response = make_request(
        'data', 
        {
            'datasetid' : 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
            'locationid' : 'CITY:US360019', # NYC
            'startdate' : current,
            'enddate' : current,
            'units' : 'metric',
            'limit' : 1000 # max allowed
        }
    )

    if response.ok:
        # we extend the list instead of appending to avoid getting a nested list
        results.extend(response.json()['results'])

    # update the current date to avoid an infinite loop
    current += datetime.timedelta(days=1)

# Save to CSV because this takes a lot of time
df = pd.DataFrame(results)
df.to_csv('data/UKtemperature.csv', index=False)


#

'Gathering data for 1998-09-12'

KeyboardInterrupt: ignored

--2020-02-05 22:02:30--  https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY
Resolving www.ncdc.noaa.gov (www.ncdc.noaa.gov)... 205.167.25.177, 205.167.25.178, 205.167.25.171, ...
Connecting to www.ncdc.noaa.gov (www.ncdc.noaa.gov)|205.167.25.177|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-02-05 22:02:31 ERROR 400: Bad Request.

